In [2]:
!pip install nbformat --quiet
!pip install --upgrade langchain --quiet
!pip install openai --quiet
!pip install python-dotenv --quiet

In [1]:
%load_ext autoreload
%autoreload 2
from MC_LIME import *
import pandas as pd

from langchain.prompts import ChatPromptTemplate
import re
from chat_open_ai import chat_openai_model

# External functions
from functions_pipeline import *
from prompt_templates import TEMPLATES

from IPython.display import display, Markdown

In [2]:
print(tf.__version__)


2.13.0


### Metavariables

In [3]:
NUM_WEEKS=5

LIME_bool=True
MC_LIME_bool=False
CEM_bool=False
average_class=False

## Import data & model
- Which student data to pick?
- Feature scores of the student
- LIME results OR MC-LIME results (depending on the strategy)

In [21]:
# CHANGE HERE THE FEATURE SET
feature_set=f"features/dsp_001_features_5_weeks.csv" 
features = pd.read_csv(feature_set)

# GET PREDICTION MODEL
model_path = 'models/'
course='dsp_001'
loaded_model=get_model(model_path, course, NUM_WEEKS)

mapping=pd.read_csv("data/user_id_mapping-dsp_001.csv", index_col=0)
students_ids=pd.read_csv("data/representative_samples.csv")["user_id"]
indices = mapping.index[mapping['user_id'].isin(students_ids)].tolist()

# Retrieve data for student idx
lime_path=f"OLD_uniform_eq_results_ori_{str(NUM_WEEKS)}_weeks/LIME/{course}/dataframes/"
cem_path=f"OLD_uniform_eq_results_ori_5_weeks/CEM/{course}/"
mapping={"Pass" : 0, "Fail" : 1}

2024-07-15 12:10:46.728698: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2024-07-15 12:10:46.949485: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2024-07-15 12:10:47.266261: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 14 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2024-07-15 12:10:47.271366: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2024-07-15 12:10:47.467576: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_sh

(None, 140)


In [22]:
# Change here the student index to test
idx=6478

# Create list of prompts 

In [23]:
template=TEMPLATES["template"]
goal_definition = TEMPLATES["goal_definition"]
features_description = get_features_description(features)
model_description = TEMPLATES["model_description"]
post_hoc_description = define_post_hoc_description(LIME_bool, MC_LIME_bool, CEM_bool)
format_instructions = TEMPLATES["format_instructions"]
course_description = TEMPLATES["course_description_dsp_001"]
data_string = get_data(course, loaded_model, idx, features, lime_path, cem_path, MC_LIME_bool, LIME_bool, CEM_bool, average_class)
conversation_template = TEMPLATES["conversation_template"]

FileNotFoundError: [Errno 2] No such file or directory: 'OLD_uniform_eq_results_ori_5_weeks/CEM/dsp_001/6478.csv'

In [ ]:
from prompt_class import prompt_class
instructions_dataset=pd.read_csv("prompting_strategies.csv")
instructions_dataset=instructions_dataset[['name', 'Communication Prompt', 'LLM-Prompt2']]
instructions_dataset = instructions_dataset[instructions_dataset['name'] != 'hesslow_contrastive']
instructions_dataset = instructions_dataset.rename(columns={'Communication Prompt': 'instruction'})
instructions_dataset

prompt_object=prompt_class(template, instructions_dataset)

context_string=prompt_object.context_partial_prompt( 
    data_string,
    features_description, 
    goal_definition, 
    model_description, 
    post_hoc_description, 
    course_description,
    )

## Explanation presentation / evaluation
In this step we give the LLM the last prompt to generate an explanation that is presented and communicated well.

In [ ]:
presentation_examples = ""

# Here if you want to change the template
presentation_template = """
Given this report, I want you to write a shorter version including: 

1. a brief description of the explanation findings 
2. Two recommended actions that the student can take.

The student who is going to interact with you is the same student that the data is about. Imagine to be a professor giving feedback to a student.

Follow the instructions underneath in the INSTRUCTIONS section. 

INSTRUCTIONS
{presentation_intruction}

Follow these rules:
- do not use any feature names, feature scores, importance scores or how the ML model works. Describe the desired behaviour instead.
- do not say the output of the model

To communicate this intervention most effectively, use Grice’s maxims of conversation.

- do not say things that you believe to be false
- do not say things for which you do not have sufficient evidence.
- do not add information that is not relevant
- Only say what is relevant
- be orderly

Your goal is to explain to the student what they should do to improve their performance in the course in the best way possible.
Follow the instructions above.

"""

# """
# FORMAT

# # Student Feedback
# ### Explanation
# [Your paragraphs here]

# ### Recommendations
# 1. **[Recommendation title]**: [Your recommendation here]
# 2. **[Recommendation title]**: [Your recommendation here]

# """

In [ ]:
prompt_object.add_explanation_template(presentation_template)
prompt_object.create_prompts_list(instructions_dataset)

,name,instruction,prompt,llm_prompt_2
0,relevance_selection,Use the social science theory “Relevance-Based...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
1,abnormal_conditions,Use the social science theory “Abnormal Condit...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
2,pearl_explanation,Use the social science theory “Pearl’s model o...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
3,necessity_robustness_selection,Use the social science theory “Necessity and R...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
4,contrastive_explanation,Use the social science theory “Contrastive Exp...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
6,base_contrastive_explanation,Use the social science theory “Contrastive Exp...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
7,statistical_relevance,Use the social science theory “Statistical Rel...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
8,chain_of_thought,Let’s think step-by-step,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."


In [ ]:
from IPython.display import display, Markdown, Latex
import warnings

presentation_prompt=prompt_object.prompting_strategies["llm_prompt_2"][0]
display(Markdown(presentation_prompt[0].content))


Given this report, I want you to write a shorter version including: 

1. a brief description of the explanation findings 
2. Two recommended actions that the student can take.

The student who is going to interact with you is the same student that the data is about. Imagine to be a professor giving feedback to a student.

Follow the instructions underneath in the INSTRUCTIONS section. 

INSTRUCTIONS
Write a section called "analysis" with sentences that follow this structure:
• Relevant Causes: Say to the student which causes you selected as relevant based on the question, context, and your background. Be explicit.
• New Information: Say to the student which information you thought they knew already, such as "Assuming that you know…" and highlight aspects that haven't been previously communicated to the student. Be clear and honest.
Finally:
• Say to the student that you focused on the most relevant causes that provide new insights. Specify which causes you selected and why they are important.

Follow these rules:
- do not use any feature names, feature scores, importance scores or how the ML model works. Describe the desired behaviour instead.
- do not say the output of the model

To communicate this intervention most effectively, use Grice’s maxims of conversation.

- do not say things that you believe to be false
- do not say things for which you do not have sufficient evidence.
- do not add information that is not relevant
- Only say what is relevant
- be orderly

Your goal is to explain to the student what they should do to improve their performance in the course in the best way possible.
Follow the instructions above.



# Testing presentation prompt with the GPT API

In [ ]:
# HERE TO TEST CHANGES FOR THE PROMPT
presentation_prompt="""
Given this report, I want you to write a shorter version including:

a brief description of the explanation findings
Two recommended actions that the student can take.
The student who is going to interact with you is the same student that the data is about. Imagine to be a professor giving feedback to a student.

Follow the instructions underneath in the INSTRUCTIONS section.

INSTRUCTIONS Write a section called "analysis" with sentences that follow this structure: • Relevant Causes: Say to the student which causes you selected as relevant based on the question, context, and your background. Be explicit. • New Information: Say to the student which information you thought they knew already, such as "Assuming that you know…" and highlight aspects that haven't been previously communicated to the student. Be clear and honest. Finally: • Say to the student that you focused on the most relevant causes that provide new insights. Specify which causes you selected and why they are important.

Follow these rules:

do not use any feature names, feature scores, importance scores or how the ML model works. Describe the desired behaviour instead.
do not say the output of the model
To communicate this intervention most effectively, use Grice’s maxims of conversation.

do not say things that you believe to be false
do not say things for which you do not have sufficient evidence.
do not add information that is not relevant
Only say what is relevant
be orderly
Your goal is to explain to the student what they should do to improve their performance in the course in the best way possible. Follow the instructions above.
"""

In [83]:
# EXPLANATION SELECTION PROMPT: DO THIS ONCE

import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Change index for changing strategy
test_prompt=prompt_object.prompting_strategies["prompt"][0]

# DEFINE LANGCHAIN MODEL
llm_class=chat_openai_model("gpt-4o", 0.5)
conversation=llm_class.define_chat_model(conversation_template)

# Here we call the model with the main prompt
result1=conversation.predict(input=test_prompt)

In [44]:
# Do this to remove the previous presentation prompt and answer to test the next one
conversation.memory.chat_memory.messages = conversation.memory.chat_memory.messages[:-2]
assert len(conversation.memory.chat_memory.messages) == 2

In [46]:
# Here we call the model with the presentation prompt
result2=conversation.predict(input=presentation_prompt)

In [38]:
display(Markdown(result1))

**Report on Student's Performance Prediction Using Necessity and Robustness Selection**

**Introduction:**
This report leverages the social science theory of "Necessity and Robustness Selection" to analyze the prediction of a student's performance in the Digital Signal Processing 1 course. The prediction model, based on a recurrent neural network, forecasts a 'pass' outcome with 75.713210% confidence. We utilize data from an explainable AI approach (LIME) and relevant student behavior data to derive insights.

**Analysis:**

1. **Defining Sets of Factors:**
   The feature importances provided by LIME highlight several key factors influencing the student's predicted performance:
   - **Competency Alignment:** Positive scores in Weeks 4 and 5 indicate that the student's ability to solve problems correctly in these weeks strongly supports a passing prediction. However, a high competency alignment in Week 3 negatively impacts the prediction.
   - **Competency Strength:** Negative scores in Weeks 3, 4, and 5 suggest that the student's efficiency in solving problems (achieving maximum grades with few attempts) in these weeks adversely affects the passing prediction.
   - **Total Clicks on Problems:** Negative scores in Weeks 4 and 5 indicate that an increase in problem-solving activity correlates negatively with the prediction, which might suggest inefficiency or struggle in these weeks.
   - **Time Between Sessions Standard Deviation:** Positive scores in Weeks 4 and 5 suggest that more regular time intervals between study sessions contribute positively to the prediction.
   - **Competency Anticipation:** Positive scores across multiple weeks indicate that the student's engagement with upcoming course content is a positive predictor of success.

2. **Selection of Necessary Factors:**
   The most necessary factors likely to cause the event (passing the course) include:
   - **Competency Alignment in Week 5:** A score of 0.213 suggests this is a strong positive predictor, indicating effective problem-solving in the critical later stage of the course.
   - **Competency Strength in Week 5:** Despite its negative impact (-0.175), this factor is crucial as it reflects on the student's approach to problem-solving.

3. **Selection of Most Robust Explanation:**
   The robustness of an explanation is determined by its stability under slightly varied conditions. The most robust factors in this analysis include:
   - **Competency Alignment in Week 5:** This factor not only has a high positive score but is also supported by a zero feature value, suggesting that even minimal problem-solving effectiveness in the final weeks is critical for passing.
   - **Time Between Sessions Standard Deviation in Week 5:** A positive impact coupled with a zero feature value indicates that maintaining consistent study intervals is crucial, regardless of their frequency or duration.

**Conclusion:**
The analysis concludes that the student's ability to effectively solve problems in the later weeks of the course (Week 5) and maintain regular study intervals are the most robust indicators of passing the course. Despite some negative impacts from competency strength, the overall prediction leans towards a pass, primarily driven by effective problem-solving and consistent study habits towards the end of the course. This insight could guide interventions aimed at enhancing student performance by focusing on consistent engagement and effective problem-solving strategies in critical weeks.

In [47]:
display(Markdown(result2))

# Student Feedback
### Explanation
In analyzing your performance in the Digital Signal Processing 1 course, several factors have emerged as particularly influential. Your ability to solve problems effectively, especially in the later weeks of the course, plays a crucial role in your overall performance. Additionally, maintaining regular intervals between your study sessions contributes significantly to your learning success. These factors are essential as they directly relate to your problem-solving skills and study habits, which are critical for mastering the course content.

Among these, the most necessary factor is your problem-solving effectiveness in the later stages of the course. This is a strong indicator of your grasp of the material and your readiness for assessments. The most robust explanation for your performance, which holds even under slightly varied conditions, is the consistency of your study intervals. This regularity in your study pattern helps ensure steady progress and better retention of the course material.

### Recommendations
1. **Enhance Problem-Solving Skills**: Focus on improving how effectively you solve problems, especially as you approach the end of the course. This might involve practicing with more complex problems or reviewing problem-solving strategies in study sessions.

2. **Maintain Consistent Study Intervals**: Try to keep a regular schedule for your study sessions. This consistency helps in managing the course load effectively and reduces the cognitive burden of last-minute learning.